In [1]:
from pprint import pprint
from gdc.data_access import *
from gdc.estimation.consumption import *

In [2]:
df_load_simulated_normalized.head()[[0, 1, 2]]

,0,1,2
2023-01-01 00:00:00,0.308875,0.241084,0.435789
2023-01-01 01:00:00,0.325552,0.246717,0.424862
2023-01-01 02:00:00,3.468337,0.245007,2.300304
2023-01-01 03:00:00,2.215840,1.272533,2.411101
2023-01-01 04:00:00,1.259481,1.027133,0.483457


In [10]:
df_load_simulated_normalized.mean().mean()

np.float32(0.80494165)

In [3]:
df_temp_simulated_normalized.head()[[0, 1, 2]]

,0,1,2
2023-01-01 00:00:00,13.021741,7.970941,7.970941
2023-01-01 01:00:00,12.621741,7.970941,7.970941
2023-01-01 02:00:00,12.621741,8.070941,8.070941
2023-01-01 03:00:00,12.421742,8.070941,8.070941
2023-01-01 04:00:00,12.321742,7.870941,7.870941


### Pooled effect model

In [4]:
p_mean_model = PooledMDHUncorrelatedErrors(
    y=df_load_simulated_normalized,
    temp=df_temp_simulated_normalized
)

beta, means = p_mean_model.fit()

In [5]:
pprint(p_mean_model.summary(beta, means, label="pooled_means"))

{'beta_CDD': 0.003547864966094494,
 'beta_HDD': 0.025797156617045403,
 'label': 'pooled_means',
 'static_fit': {'r2': 0.09677087276703245, 'rmse': 0.8561899656671869}}


### Fixed effect model

In [6]:
i_mean_model = IndividualMDHUncorrelatedErrors(
    y=df_load_simulated_normalized,
    temp=df_temp_simulated_normalized
)

beta, means = i_mean_model.fit()

In [7]:
pprint(i_mean_model.summary(beta, means, label="iMDH_means"))

{'beta_CDD': 0.002452349755913019,
 'beta_HDD': 0.030275173485279083,
 'label': 'iMDH_means',
 'static_fit': {'r2': 0.533043806097159, 'rmse': 0.6156150617453132}}


In [8]:
error_ar1_24 = ARErrorModel(
    i_mean_model.static_resids(beta, means), lags=(1, 24))
error_ar1_24_res = error_ar1_24.fit()
summary = i_mean_model.summary(
            beta, means, error_model_=error_ar1_24_res,
            label="IndividualMDH with AR(1, 24) errors")
pprint(summary)

{'beta_CDD': 0.002452349755913019,
 'beta_HDD': 0.030275173485279083,
 'dynamic_fit': {'coeffs': [0.25199447309015743, 0.44859790182903275],
                 'lags': [1, 24],
                 'r2': 0.6709014815856378,
                 'rmse': 0.516813833857603},
 'label': 'IndividualMDH with AR(1, 24) errors',
 'static_fit': {'r2': 0.533043806097159, 'rmse': 0.6156150617453132}}
